In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
train_df = pd.read_csv('./dataset/final/train.csv', usecols=['source', 'label'])
train_df['source'] = train_df['source'].str.lower()
train_df['label'] = train_df['label'].str.lower()
train_ds = Dataset.from_pandas(train_df)

test_df = pd.read_csv('./dataset/final/test.csv', usecols=['source', 'label'])
test_df['source'] = test_df['source'].str.lower()
test_df['label'] = test_df['label'].str.lower()
test_ds = Dataset.from_pandas(test_df)

dev_df = pd.read_csv('./dataset/final/dev.csv', usecols=['source', 'label'])
dev_df['source'] = dev_df['source'].str.lower()
dev_df['label'] = dev_df['label'].str.lower()
dev_ds = Dataset.from_pandas(dev_df)

all_df = pd.concat((train_df, test_df, dev_df))
all_ds = Dataset.from_pandas(all_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./results', local_files_only=True)
model = AutoModelForSeq2SeqLM.from_pretrained('./results', local_files_only=True)

In [ ]:
a = 0
t = 0

for source, label in zip(test_df['source'], test_df['label']):
    input_ids = tokenizer(source, return_tensors="pt", max_length=256, padding=True, truncation=True).input_ids 
    outputs = model.generate(input_ids)
    
    decoded = ''.join(tokenizer.convert_ids_to_tokens(outputs[0])[1:-1]).replace('▁', ' ').strip()
    
    t += 1

    if decoded == label:
        a += 1
    else:
        print(source)
        print(decoded)
        print(label)
        print()